In [ ]:
'''Need to calculate the following:
Classifier:
P(s|T) = (P(s)*P(T|s))/P(T)
T is the text of the tweet and s is the sentiment

P(T|s) = P(G|s) = product(for every g in G)(P(g|s))
Where G is the set of n-grams and g is an individual n-gram
Calculate P(g|s) using +1 for 0 values

Calculate P(T) for all T
Then add all g to sets s with counts
'''


#haven't collected corpus yet, assume this will be filled in
#LOAD PROCESSED TWEETS HERE 
hillTweets
trumpTweets

#Load tokens files for bigrams
with open('hill_tokens.txt') as f:
    hillTokensFile = f.read()
    
hillTokens = hillTokensFile.strip().split('\n')

with open('trump_tokens.txt') as f:
    trumpTokensFile = f.read()
    
trumpTokens = trumpTokensFile.strip().split('\n')

#LOAD COUNTS OF TYPES IN EACH HERE AS HILLTYPES, BOTHTYPES, AND TRUMPTYPES

[0.9892174006000001, 0.0034346941000000003]

In [ ]:
#for every word, add it to the dictionary in this format:
#key is the word, value is another dictionary
    #within each subdictionary, have the following word as the key
    #and the number of times it follows as the value

#make a bigrams dictionary for both and each candidate
bigrams = {}
hillBigrams = {}
trumpBigrams = {}

#def generateBigramsDict():
#Add all Hillary positive bigrams to both hillBigrams and bigrams
for i in range(len(hillTokens) - 1):
    #if the word already exists
    if hillTokens[i] in hillBigrams:
        #if the following word already exists
        if hillTokens[i+1] in hillBigrams[hillTokens[i]]:
            #add 1 to the number of occurences
            hillBigrams[hillTokens[i]][hillTokens[i+1]] += 1
            bigrams[hillTokens[i]][hillTokens[i+1]] += 1
        #add it to the subdictionary of hill
        else:
            hillBigrams[hillTokens[i]][hillTokens[i+1]] = 1
            bigrams[hillTokens[i]][hillTokens[i+1]] = 1
    #the word does not exist yet in hillBigrams
    else:
        #initialize a new subdictionary and add the first bigram
        hillBigrams[hillTokens[i]] = {hillTokens[i+1]: 1}
        bigrams[hillTokens[i]] = {hillTokens[i+1]: 1}

#Add all Trump positive bigrams to both trumpBigrams and bigrams
for i in range(len(trumpTokens) - 1):
    #if the word already exists
    if trumpTokens[i] in trumpBigrams:
        #if the following word already exists
        if trumpTokens[i+1] in trumpBigrams[trumpTokens[i]]:
            #add 1 to the number of occurences
            trumpBigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
            bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
        #add it to the subdictionary of trump
        else:
            trumpBigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
            #check bigrams
            if trumpTokens[i+1] in bigrams[trumpTokens[i]]:
                bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
            else:
                bigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
    #the word does not exist yet in trumpBigrams
    else:
        #initialize a new subdictionary and add the first bigram
        trumpBigrams[trumpTokens[i]] = {trumpTokens[i+1]: 1}
        #check bigrams
        if trumpTokens[i] in bigrams:
            #check word
            if trumpTokens[i+1] in bigrams[trumpTokens[i]]:
                bigrams[trumpTokens[i]][trumpTokens[i+1]] += 1
            else:
                bigrams[trumpTokens[i]][trumpTokens[i+1]] = 1
        else:
            bigrams[trumpTokens[i]] = {trumpTokens[i+1]: 1}
            


In [ ]:
from numpy import float32
from math import log, exp

'''Need to implement Laplace smoothing'''


'''MLE = count of bigram / count of first unigram
Laplace: P = (count of bigram + 1)/(count of first unigram + V)
where V is the number of possible bigrams given unigram
What do we do if the given unigram doesn't exist?'''

'''Assume this returns log probabilities'''
def smoothProb(word1, word2, sentiment):
    P = numpy.float32(0.0)
  
    #True = hillary
    if sentiment == True:
        #add one for smoothing
        unigramCount = hillTypes[word1] + 1
        #if the unigram has been seen before
        if word1 in hillBigrams.keys():
            #if this bigram has been seen before
            if word2 in hillBigrams[word1].keys():
                P = (hillBigrams[word1][word2] + 1) / (unigramCount + len(hillBigrams[word1]))
                #maybe log it? FIGURE OUT LATER
            #the second word has not followed the first word before
            else:
                P = 1 / (unigramCount + len(hillBigrams[word1]))
        #this unigram has not been seen before
        else:
            P = unigramCount / (len(hillTokens)+len(hillTypes))
        return P
    #False = trump
    else:
        #add one for smoothing
        unigramCount = trumpTypes[word1] + 1
        #if the unigram has been seen before
        if word1 in trumpBigrams.keys():
            #if this bigram has been seen before
            if word2 in trumpBigrams[word1].keys():
                P = (trumpBigrams[word1][word2] + 1) / (unigramCount + len(trumpBigrams[word1]))
                #maybe log it? FIGURE OUT LATER
            #the second word has not followed the first word before
            else:
                P = 1 / (unigramCount + len(trumpBigrams[word1]))
        #this unigram has not been seen before
        else:
            P = unigramCount / (len(trumpTokens)+len(trumpTypes))
        return P
    
    

In [ ]:
'''Need to calculate the following:
Classifier:
P(s|T) = (P(s)*P(T|s))/P(T)
T is the text of the tweet and s is the sentiment

P(T|s) = P(G|s) = product(for every g in G)(P(g|s))
Where G is the set of n-grams and g is an individual n-gram
Calculate P(g|s) using +1 for 0 values

Calculate P(T) for all T
Then add all g to sets s with counts
'''

#probS is number of tweets in that sentiment divided by total number of tweets

def classify(tweet):
    #calculate probT
    probT = tweetCount(tweet)/len(tweetCount)
    #probS is number of tweets in that sentiment divided by total number of tweets
    probS = len(hillTweets)/(len(hillTweets) + len(trumpTweets))
    